# Baseline Model

In [ ]:
from openai import OpenAI
import numpy as np

client = OpenAI(api_key)

In [ ]:
def diagnose(prompt):
    messages = [
        {"role": "system", "content": "You are a helpful doctor, only output a list" },
        {"role": "user", "content": prompt}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages = messages
    )
    return response.choices[0].message.content

# Test

In [ ]:
import pandas as pd
from nltk.stem import PorterStemmer
import numpy as np

ps = PorterStemmer()

df = pd.read_csv('/content/sampled_df.csv')
df.head()

,output_text,input_text,sentence_1,sentence_2,sentence_3,sentence_4,sentence_5,sentence_6,sentence_7
0,allergy,"My skin is itchy and red. Sometimes, it can pe...",My skin is itchy and red.,"Sometimes, it can peel.","My lips and cheeks swell, and it is really ann...",I sometimes get headaches and watery eyes beca...,no more information,NaN,NaN
1,allergy,I have a sore throat and I am sneezing all the...,I have a sore throat and I am sneezing all the...,"Sometimes my face swells up, especially my lip...",I can't stop sneezing once I start.,no more information,NaN,NaN,NaN
2,allergy,"I feel sick to my stomach, lightheaded, and di...","I feel sick to my stomach, lightheaded, and di...",My throat is swollen and I can't breathe well.,I sometimes feel pain in my chest and nausea a...,no more information,NaN,NaN,NaN
3,arthritis,I've been feeling really weak in my muscles an...,I've been feeling really weak in my muscles an...,My joints have been swelling up and it's hard ...,Walking has been really painful too.,no more information,NaN,NaN,NaN
4,arthritis,"I've been having a lot of pain in my joints, e...","I've been having a lot of pain in my joints, e...",It's hard to move around and I'm always feelin...,I've also been having trouble sleeping because...,no more information,NaN,NaN,NaN


In [ ]:
diagnoses = []
format = "\nGive me a list of 4 possible illnesses. The output should in format: '\nIllness List:\n    1. Disease_1\n    2. Disease_2\n    3. Disease_3\n    4. Disease_4'"

In [ ]:
for i in range(0, 66):
    print(f'Patient {i+1}')

    user_input = df.iloc[i]['input_text']
    print(f'Patient: {user_input}')

    response = diagnose(user_input + format)

    print(f'Doctor: {response}')
    diagnoses.append(response)

    print('\n------------------------------\n')

Patient 1
Patient: My skin is itchy and red. Sometimes, it can peel. My lips and cheeks swell, and it is really annoying. I sometimes get headaches and watery eyes because of the swelling.
Doctor: Illness List:
    1. Contact Dermatitis
    2. Allergic Reaction
    3. Eczema (Atopic Dermatitis)
    4. Angioedema

------------------------------

Patient 2
Patient: I have a sore throat and I am sneezing all the time. Sometimes my face swells up, especially my lips and the area around my eyes. I can't stop sneezing once I start.
Doctor: Illness List:
    1. Allergic Rhinitis
    2. Viral Upper Respiratory Infection (Common Cold)
    3. Sinusitis
    4. Angioedema

------------------------------

Patient 3
Patient: I feel sick to my stomach, lightheaded, and dizzy. My throat is swollen and I can't breathe well. I sometimes feel pain in my chest and nausea at night.
Doctor: Illness List:
    1. Anaphylaxis
    2. Gastroesophageal Reflux Disease (GERD)
    3. Panic Attack
    4. Upper Respir

In [ ]:
df_output = pd.DataFrame(data={'disease':df['output_text'], 'diagnoses':diagnoses, 'description':df['input_text']})
df_output.to_csv('output.csv')

In [ ]:
print(len(diagnoses))

66


In [ ]:
# Successful Rate Evaluation
scores = np.array([0.0] * 66)
for i in range(66):
    for diagnosis in diagnoses[i].split('\n')[1:]:
        # Check if the diagnosis list contains the disease
        if ps.stem(df['output_text'][i]) in diagnosis.lower():
             scores[i] = 1.0
             break

In [ ]:
print(f'Score: {scores.sum()/66}')

Score: 0.5454545454545454


In [ ]:
# Score-Based Evaluation
points = np.array([0.0] * 66)
for i in range(66):
    rank = 1
    for diagnosis in diagnoses[i].split('\n')[1:]:
        # Check if the diagnosis list contains the disease
        if ps.stem(df['output_text'][i]) in diagnosis.lower() and rank == 1:
             points[i] = 5.0
             break
        elif ps.stem(df['output_text'][i]) in diagnosis.lower() and rank == 2:
             points[i] = 3.0
             break
        elif ps.stem(df['output_text'][i]) in diagnosis.lower() and rank == 3:
             points[i] = 2.0
             break
        elif ps.stem(df['output_text'][i]) in diagnosis.lower() and rank == 4:
             points[i] = 1.0
             break

        rank += 1

print(f'Score: {points.sum()/(66*5.0)}')

Score: 0.45454545454545453
